<a href="https://colab.research.google.com/github/wiiiiish/2022-machine_learning/blob/main/s_pred_model_0504.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!ls

drive  sample_data


In [2]:
%cd drive/MyDrive/Colab\ Notebooks/2022-04-27\ 데이터

/content/drive/MyDrive/Colab Notebooks/2022-04-27 데이터


In [4]:
!sudo apt-get install python3-dev default-libmysqlclient-dev
!pip install pymysql

Reading package lists... Done
Building dependency tree       
Reading state information... Done
default-libmysqlclient-dev is already the newest version (1.0.4).
default-libmysqlclient-dev set to manually installed.
python3-dev is already the newest version (3.6.7-1~18.04).
python3-dev set to manually installed.
The following packages were automatically installed and are no longer required:
  libnvidia-common-460 nsight-compute-2020.2.0
Use 'sudo apt autoremove' to remove them.
0 upgraded, 0 newly installed, 0 to remove and 42 not upgraded.
     |████████████████████████████████| 43 kB 1.2 MB/s 


In [5]:
import pandas as pd
import pymysql

In [6]:
from pymysql.converters import convert_bit
host = "project-db-stu.ddns.net"
port = 3307
username = "wish"
password = "1234"
db = 'wish'

conn = pymysql.connect(host=host, port=port, user=username, password=password, db=db)

In [18]:
from sklearn.linear_model import LinearRegression # 회귀 모델

In [19]:
s_pred_model = LinearRegression()

In [33]:
weather = pd.read_csv('./data/weather_0504.csv', encoding='utf-8')
weather

,date_time,temperature,precipitation,wind_speed,humidity,radiation,total_cloudiness,visibility,ground_temp
0,60010,18.393009,0,0.626055,99.077047,0.00000,5.383689,1406.307589,20.587329
1,60020,18.123292,0,0.666385,98.931846,0.00000,5.378191,1520.985931,20.922063
2,60030,18.382198,0,0.640155,98.910705,0.00000,5.807724,1583.682653,20.663506
3,60040,18.113827,0,0.706955,98.936457,0.00000,5.654719,1854.592384,21.152932
4,60050,18.202366,0,0.633896,98.969637,0.00000,7.202626,1839.823058,20.929866
...,...,...,...,...,...,...,...,...,...
1075,85920,18.945224,0,0.097051,98.992754,0.00000,8.748208,1912.489642,23.716064
1076,85930,18.927292,0,0.258281,98.922693,0.00000,10.301976,1863.227213,22.459806
1077,85940,18.909723,0,0.308918,98.990245,0.00000,6.605528,1884.812965,22.512350
1078,85950,19.188160,0,0.256762,98.961152,0.00000,9.495882,1884.334552,23.560265


In [34]:
aod = pd.read_csv('./data/aod_0504.csv', encoding='utf-8')
aod

,Time - Plot 0,Power(kW) - Plot 0
0,07:00:10,10.4142
1,07:00:20,10.7484
2,07:00:30,11.0826
3,07:00:40,11.4168
4,07:00:50,11.7510
...,...,...
1075,09:59:20,234.1310
1076,09:59:30,234.0980
1077,09:59:40,234.0660
1078,09:59:50,234.0330


In [35]:
# 문제와 답 분리
X = weather[['date_time', 'temperature', 'precipitation', 'wind_speed', 'humidity', 'radiation', 'total_cloudiness', 'visibility', 'ground_temp']]
y = aod['Power(kW) - Plot 0']

In [36]:
# 훈련용과 평가용으로 데이터 분리
# 비율은 8:2, randomstate = 504
from sklearn.model_selection import train_test_split # 훈련용, 평가용으로 데이터 분리

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=504)
X_train.head()

,date_time,temperature,precipitation,wind_speed,humidity,radiation,total_cloudiness,visibility,ground_temp
487,72120,18.607171,0,0.188708,99.022224,0.0,5.607489,1874.384759,21.486196
351,65840,18.429063,0,0.625102,99.020107,0.0,6.989410,1916.657663,20.027758
840,82010,18.962048,0,0.318916,99.026021,0.0,8.694109,1866.711442,22.204734
452,71530,18.826980,0,0.525701,99.026600,0.0,6.094614,1906.411195,21.675183
154,62550,18.077350,0,0.693414,98.965130,0.0,6.962360,1616.185930,20.992935


In [37]:
s_pred_model.fit(X_train, y_train)  # 지도학습 방법 (회귀)

LinearRegression()

In [41]:
# 선형 모델이 입력 특성에 대해 학습한 가중치
s_pred_model.coef_

array([ 1.05507280e-02, -6.51981954e+00, -2.78888024e-13, -8.24731949e+00,
       -6.48316378e+00, -5.08634019e+02, -1.05400045e+01,  3.01180267e-02,
       -8.76604434e+00])

In [42]:
# 절편
s_pred_model.intercept_

370.84706185750724

In [44]:
# 모델 
s_pre = s_pred_model.predict(X_test)

In [45]:
from sklearn.metrics import mean_absolute_error # 평균 절대값 오차

In [46]:
s_pred_model.score(X_test, y_test) # 평가용 문제, 평가용 답

0.9188698868281694

In [86]:
!pip install joblib

In [87]:
# 모델 저장
import pickle
with open('s_model.pickle', 'wb') as f:
    pickle.dump(s_pred_model, f, pickle.HIGHEST_PROTOCOL)

In [88]:
with open('s_model.pickle', 'rb') as f:
    new_model = pickle.load(f)

In [89]:
new_model.predict(X_test)

array([257.42762593, 197.29113573, 219.71791131, 204.27771486,
       260.60117023, 195.94469588, 102.51395712, 167.90683324,
       252.82885858, 165.74533053, 233.67669186,  64.55266503,
       109.43165154, 177.14296342, 272.59309268, 219.84212345,
       262.94310794, 173.7322545 , 154.19291105,  70.75179258,
       241.33512025, 209.19907132, 193.60735286, 158.64444716,
        84.07632357, 275.10355688, 173.26719142, 262.72000709,
       246.04721551, 267.43283991,  61.05757579, 103.37471303,
        70.26135334, 200.92639332, 184.53513522, 239.82766644,
       269.4608716 , 205.8963807 , 239.53449033, 205.63047749,
        41.71629738,  61.87249615,  49.1807198 ,  84.2152312 ,
       245.77456919, 205.61230373, 176.2013712 ,  41.72282551,
       180.78489366,  68.04888342, 268.65628062, 183.44464374,
       220.72430439, 169.64781155, 239.86776599, 265.80517978,
        65.8550563 , 205.20339169, 172.70889952,  91.32552766,
       218.83372207, 152.76689734, 278.61700111, 224.02

In [70]:
test = pd.read_csv('./data/weather_050409.csv', encoding='utf-8')
test

,date_time,temperature,precipitation,wind_speed,humidity,radiation,total_cloudiness,visibility,ground_temp
0,90000,20.938822,0,0.300982,96.912220,0.109630,10.010055,1999.981606,24.878155
1,90010,20.294810,0,0.358722,92.907333,0.070291,9.954438,2000.023331,27.982545
2,90020,20.257064,0,0.407594,98.447857,0.116916,9.974488,1999.983233,26.390490
3,90030,19.652657,0,0.444139,93.621594,0.094525,9.950634,1999.990252,26.321891
4,90040,20.635626,0,0.298696,93.618658,0.088545,10.017615,2000.055380,24.784456
...,...,...,...,...,...,...,...,...,...
356,95920,19.880623,0,0.492466,93.381098,0.040413,9.930506,1999.906247,27.752585
357,95930,20.276046,0,0.387107,92.974195,0.012187,9.953285,2000.054798,26.961847
358,95940,20.520093,0,0.375380,93.198160,-0.014780,10.025929,2000.026368,25.958624
359,95950,20.072582,0,0.422769,97.585123,0.048698,10.046478,1999.953497,27.816882


In [71]:
s_test = s_pred_model.predict(test)

In [81]:
s_ans = pd.read_csv('./data/aod_050410.csv', encoding='utf-8')

In [76]:
s_ans = ans['Power(kW) - Plot 0']
s_ans

0      233.825
1      233.651
2      233.476
3      233.302
4      233.127
        ...   
355    171.838
356    171.664
357    171.489
358    171.315
359    171.140
Name: Power(kW) - Plot 0, Length: 360, dtype: float64

In [83]:
s_test

array([234.        , 257.17599782, 211.23292817, 258.50877184,
       269.23890065, 286.3761158 , 240.29943086, 302.71089274,
       262.7278005 , 249.16739505, 270.05633522, 252.80187636,
       269.84066178, 298.65324049, 267.99986933, 231.9397543 ,
       273.88188849, 237.42112985, 252.38099135, 247.07369775,
       283.87911814, 317.34019046, 231.09147735, 238.95379115,
       241.87885352, 291.47173792, 300.51578491, 248.27764351,
       293.42039223, 290.03286096, 311.25643921, 272.2335201 ,
       277.87507298, 278.51710691, 314.90267998, 275.96769075,
       274.09449354, 310.74041731, 276.21505226, 277.59378603,
       291.92311366, 288.31053989, 285.33251948, 245.18084874,
       279.91797698, 289.25362481, 256.35646779, 326.2735183 ,
       284.42662359, 285.97835516, 271.3875896 , 293.68857824,
       264.98845612, 248.42017163, 280.29398232, 301.04954276,
       288.27075891, 229.71405476, 235.73252189, 292.30615847,
       298.40047049, 281.12601469, 237.5093613 , 288.56

In [91]:
new_test = new_model.predict(X_test[0:1])

In [92]:
new_test

array([257.42762593])